<a href="https://colab.research.google.com/github/MuhammadIrzam447/visionCodes/blob/master/Train_ViT_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown https://drive.google.com/uc?id=14u9sgibcvl5-2oThkjYWlgYPvgjp0RhV

Downloading...
From: https://drive.google.com/uc?id=14u9sgibcvl5-2oThkjYWlgYPvgjp0RhV
To: /content/imgs-train+test.zip
100% 1.36G/1.36G [00:15<00:00, 89.2MB/s]


In [2]:
!unzip "/content/imgs-train+test.zip"

Streaming output truncated to the last 5000 lines.
  inflating: img-train+test/train/0/74368.jpg_4.png  
  inflating: img-train+test/train/0/74380.jpg_4.png  
  inflating: img-train+test/train/0/74385.jpg_4.png  
  inflating: img-train+test/train/0/74391.jpg_4.png  
  inflating: img-train+test/train/0/74502.jpg_4.png  
  inflating: img-train+test/train/0/74509.jpg_4.png  
  inflating: img-train+test/train/0/74531.jpg_4.png  
  inflating: img-train+test/train/0/74598.jpg_4.png  
  inflating: img-train+test/train/0/74602.jpg_4.png  
  inflating: img-train+test/train/0/74612.jpg_4.png  
  inflating: img-train+test/train/0/74615.jpg_4.png  
  inflating: img-train+test/train/0/74621.jpg_4.png  
  inflating: img-train+test/train/0/74630.jpg_4.png  
  inflating: img-train+test/train/0/74631.jpg_4.png  
  inflating: img-train+test/train/0/74650.jpg_4.png  
  inflating: img-train+test/train/0/74658.jpg_4.png  
  inflating: img-train+test/train/0/74825.jpg_4.png  
  inflating: img-train+test/tra

In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00


In [4]:
import torch
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score
from PIL import Image

In [5]:
# Define the path to your training and validation data
train_data_root = "/content/img-train+test/train"
val_data_root = "/content/img-train+test/test"

In [6]:
# from transformers import ViTImageProcessor

# processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
# image_mean, image_std = processor.image_mean, processor.image_std
# size = processor.size["height"]

# # Define transformations for the input images
# transform = transforms.Compose([
#     transforms.Resize(size),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=image_mean, std=image_std)
# ])


In [7]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.image_folder = ImageFolder(root_dir, transform=None)

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, idx):
        image_path, label = self.image_folder.imgs[idx]
        image = self.load_image(image_path)
        return image, label

    def load_image(self, path):
        image = Image.open(path).convert("RGB")
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
        image = transform(image)
        return image

In [8]:
# Load the dataset using ImageFolder and apply transformations
train_dataset = ImageDataset(train_data_root)
val_dataset = ImageDataset(val_data_root)

In [9]:
# Create label2id and id2label dictionaries based on the class names in the dataset
label2id = {class_name: idx for class_name, idx in train_dataset.image_folder.class_to_idx.items()}
id2label = {idx: class_name for class_name, idx in label2id.items()}

In [37]:
def custom_collate(batch):
    print("item1: ", batch[0])
    print("item2: ", batch[1])

    images = [feature_extractor(images=item[0], return_tensors="pt") for item in batch]
    # images = [item['pixel_values'] for item in images]
    labels = [torch.tensor(item[1]) for item in batch]

    # images, labels = zip(*batch)
    # images = zip(*images)

    # images = torch.stack(images, dim=0)
    images = torch.stack([example['pixel_values'] for example in images])
    labels = torch.tensor(labels)

    print(f"images.shape {images.shape}")
    print(f"labels.shape {labels.shape}")
    print("images: ", images)

    return images, labels

In [38]:
# Initialize the feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# Define batch size and number of workers (adjust based on your system's resources)
batch_size = 4
num_workers = 1

# Create DataLoader for the dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn= custom_collate)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn= custom_collate)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [12]:
num_classes = len(train_dataset.image_folder.classes)
print(num_classes)

2


In [13]:
vit = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", id2label=id2label, label2id=label2id, num_labels=num_classes)
vit.classifier = nn.Linear(vit.config.hidden_size, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit.to(device)
print(vit)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(vit.parameters(), lr=1e-5)
# optimizer = optim.SGD(vit.parameters(), lr=0.001, momentum=0.9)
num_epochs = 20

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [39]:
for epoch in range(num_epochs):
    vit.train()
    train_loss = 0.0

    for images, labels in train_loader:
        # print(f"images.shape {images.shape}")
        # print(f"labels.shape {labels.shape}")
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = vit(images)
        loss = criterion(outputs.logits, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    # Calculate average loss for this epoch
    train_loss /= len(train_loader.dataset)

    save_dir = "/content/Train-ViT-02/"
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

    model_name = str(epoch+1) + "_model.pth"
    save_path = os.path.join(save_dir, model_name)  # Specify the complete path to the model file
    torch.save(vit.state_dict(), save_path)

    # Validation
    vit.eval()
    val_loss = 0.0
    correct = 0
    predicted_classes = []
    actual_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = vit(images)
            val_loss += criterion(outputs.logits, labels).item() * images.size(0)

            probabilities = torch.softmax(outputs.logits, dim=1)
            predicted = torch.argmax(probabilities, dim=1)

            correct += (predicted == labels).sum().item()

            predicted_classes.extend(predicted.cpu().numpy())
            actual_labels.extend(labels.cpu().numpy())

    # Calculate average loss and accuracy for validation set
    val_loss /= len(val_loader.dataset)
    accuracy = correct / len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {train_loss:.4f} - Validation Loss: {val_loss:.4f} - Accuracy: {accuracy:.4f}")

    # Compute evaluation metrics using the predicted_classes and actual_labels lists
    accuracy = accuracy_score(actual_labels, predicted_classes)
    precision = precision_score(actual_labels, predicted_classes, average='weighted')
    recall = recall_score(actual_labels, predicted_classes, average='weighted')
    f1 = f1_score(actual_labels, predicted_classes, average='weighted')

    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    print(classification_report(actual_labels, predicted_classes))
    print("Confusion Matrix:")
    print(confusion_matrix(actual_labels, predicted_classes))
    print("AUROC:", roc_auc_score(actual_labels, predicted_classes))



item1:  (tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]]), 0)
item2:  (tensor([[[0.7137, 0.7098, 0.6941,  ..., 0.4941, 0.5098, 0.5098],
         [0.7804, 0.7804, 0.7804,  ..., 0.4941, 0.5059, 0.5059],
         [0.7765, 0.7804, 0.7804,  ..., 0.4784, 0.4627, 0.4510

ValueError: ignored